<a href="https://colab.research.google.com/github/jmestanza/natural-language-processing-practice/blob/main/desafios/Desafio_3/Desafio_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Desafío 3
Entrenar un modelo de lenguaje basado en caracteres utilizando la [notebook](https://github.com/jmestanza/procesamiento_lenguaje_natural/blob/main/clase_3/ejercicios/3_modelo_lenguaje_char.ipynb) de clase como base .

Objetivos principales
- Comprender el modelo de lenguaje basado en caracteres.
- Definir un corpus para entrenar el modelo.
- Tomar decisiones sobre la arquitectura del modelo.
- Ejecutar el entrenamiento y analizar los resultados.
- Generar secuencias de texto utilizando el modelo entrenado.

Descripción de la tarea
Se proporciona un notebook base que trabaja con el libro La vuelta al mundo en 80 días.

Se realiza preprocesamiento del texto, eliminando etiquetas HTML y convirtiendo todo a minúsculas para reducir el vocabulario.

Se define un vocabulario de caracteres, incluyendo letras, signos de puntuación y espacios.

Se crean diccionarios de tokenización (carácter a índice y viceversa).

Se divide el texto en conjuntos de entrenamiento y validación, tomando el 10% del texto para validación.

El modelo se entrena en un esquema many-to-many, donde la entrada es una secuencia de caracteres y la salida es la misma secuencia desplazada en un carácter.

Se menciona el uso de capa embedding y time-distributed layers para manejar secuencias.

Se mide la perplejidad del modelo como métrica de evaluación.

Generación de texto
Se propone experimentar con diferentes estrategias de generación de secuencias.

Se pueden probar enfoques deterministas o estocásticos para generar texto.

Se debe documentar los resultados de generación obtenidos con el modelo final.

Entrega esperada
Modelo de lenguaje entrenado con la arquitectura seleccionada.

Ejemplos de generación de texto con el modelo.

Reflexión sobre los resultados y ajustes realizados.

En resumen, el trabajo implica entrenar un modelo de lenguaje basado en caracteres, ajustar su arquitectura, medir su rendimiento y generar texto con él.